# Imports

In [1]:
import os
import numpy as np
os.environ["SAGE_NUM_THREADS"] = '64' #set to number of cores

In [2]:
run create_generic_polynomial_ICML.ipynb

to create a generic polynomial with a given number of indeterminates with generic coefs

In [3]:
run get_all_polynomial_conserved_functions_ICML.ipynb 

to get all polynomial conservation laws for given vector fields as presented in section 3.1.1

In [29]:
run get_specific_vector_fields_ICML.ipynb

to get 
- the vector fields associated to the study of linear and ReLU neural networks in Momentum Flow with a constant $\tau$ or with $\tau(t):=3/t$ (Euclidean Heavy-ball and Euclidean Nesterov scenarios)
- the vector fields associated to the study of the NMF Problem with mirror geometry on both $U$ and $V$ (in the gradient flow and in the momentum scenario).
- the vector fields associated to the study of the ICNN Problem with a $2$-layer ReLU network $U \sigma (Vx + b)$ with a mirror geometry on $U$ and a euclidean metric on $V$ and $b$ (in the gradient flow and in the momentum scenario).

In [5]:
run computation_dimension_trace_Lie_algebra_ICML.ipynb # to get the number of independent conservation laws

In [6]:
sage.parallel.ncpus.ncpus()

64

In [7]:
# increase the maximum memory usage
# warning: this can consume a lot of RAM
maxima._eval_line(":lisp (ext:set-limit 'ext:heap-size 100000000000)",
wait_for_prompt=False)

**In this notebook we test our results on different vector fields $(\chi_i)_{i}$.**

### Table of contents: 
- **Part A: Test on the Euclidean Metric case, with momentum for linear (See Section 4.1.1 of our paper) and ReLU neural networks (See 4.1.2 of our paper)**  
- **Part B: Test on the vector fields associated to the NMF problem (See Section 4.2.1 of our paper)**  
- **Part C: Test on the vector fields associated to the ICNN problem (see Section 4.2.2 of our paper)**  

# A. Euclidean Metric (with momentum) for linear and ReLU neural networks

In [8]:
from tqdm import tqdm

In [33]:
np.random.seed(0)
for i in tqdm(range(20)):
    depth = np.random.randint(2, 4)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(1, 4)
    list_dim = list(list_dim)
    tau_value = np.random.randint(0, 1)
    D, vector_fields = vector_fields_for_q_layer_LNN_momentum(list_dim, tau_value)
    dim_found = D - dim_trace_lie_algebra(vector_fields, verbose=0)
    degree = 2 + tau_value
    dim_poly = solve(degree, vector_fields)
    assert dim_found == dim_poly


100%|███████████████████████████████████████████| 20/20 [02:19<00:00,  6.96s/it]


The number of independent functions found by algorithm presented in section 3.1.1 matches with $2D+1 - \text{dim} Lie W_{\phi} (\alpha)$: there is no other conservation law!

In [11]:
np.random.seed(0)
for i in tqdm(range(20)):
    depth = np.random.randint(2, 4)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(1, 4)
    list_dim = list(list_dim)
    tau_value = np.random.randint(0, 1)
    if np.random.random() < 0.5:
        bias = True
    else:
        bias = False
    D, vector_fields = vector_fields_ReLU_q_layers_momentum(list_dim, bias, tau_value)
    dim_found = D - dim_trace_lie_algebra(vector_fields, verbose=0)
    assert dim_found == 0 

100%|███████████████████████████████████████████| 20/20 [01:03<00:00,  3.17s/it]


For the ReLU case in the Euclidean momentum case, we obtain that there is zero conservation law!

# B. An example of non-Euclidean metric: the NMF

Gradient Flow case.

In [13]:
np.random.seed(0)
for i in tqdm(range(20)):
    list_dim = np.empty(3, dtype=int)
    for j in range(3):
        list_dim[j] = np.random.randint(1,8)
    list_dim = list(list_dim)
    D, vector_fields = NMF(list_dim[0], list_dim[1], list_dim[2])
    dim_found = D - dim_trace_lie_algebra(vector_fields, verbose=0)
    dim_poly = solve(2, vector_fields, False, True)
    assert dim_found == dim_poly

100%|███████████████████████████████████████████| 20/20 [07:28<00:00, 22.42s/it]


The number of independent functions found by algorithm presented in section 3.1.1 matches with $D - \text{dim} Lie W_{\phi} (\theta)$: there is no other conservation law!

Momentum flow case.

In [18]:
np.random.seed(0)
for i in tqdm(range(20)):
    list_dim = np.empty(3, dtype=int)
    for j in range(3):
        list_dim[j] = np.random.randint(1,6)
    list_dim = list(list_dim)
    tau_value = np.random.randint(0, 1)
    D, vector_fields = NMF_momentum(list_dim[0], list_dim[1], list_dim[2], tau_value)
    dim_found = D - dim_trace_lie_algebra(vector_fields, verbose=0)
    assert dim_found == 0


100%|███████████████████████████████████████████| 20/20 [01:45<00:00,  5.25s/it]


Again, in this mirror geometry with momentum, we obtain that there is zero conservation law!

# C. Another non-Euclidean Case: ICNN.

The Gradient flow case.

In [20]:
np.random.seed(0)
for i in tqdm(range(20)):
    list_dim = np.empty(3, dtype=int)
    for j in range(3):
        list_dim[j] = np.random.randint(1,6)
    list_dim = list(list_dim)
    if np.random.random() < 0.5:
        bias = True
    else:
        bias = False
    D, vector_fields = ICNN(list_dim[0], list_dim[1], list_dim[2], bias)
    dim_found = D - dim_trace_lie_algebra(vector_fields, verbose=0)
    dim_poly = solve(2, vector_fields, False, True)
    assert dim_found == dim_poly

100%|███████████████████████████████████████████| 20/20 [12:36<00:00, 37.83s/it]


The number of independent functions found by algorithm presented in section 3.1.1 matches with $D- \text{dim} Lie W_{\phi} (\theta)$: there is no other conservation law!

The Momentum flow case.

In [32]:
np.random.seed(0)
for i in tqdm(range(20)):
    list_dim = np.empty(3, dtype=int)
    for j in range(3):
        list_dim[j] = np.random.randint(1,6)
    list_dim = list(list_dim)
    if np.random.random() < 0.5:
        bias = True
    else:
        bias = False
    tau_value = np.random.randint(0, 1)
    D, vector_fields = ICNN_momentum(list_dim[0], list_dim[1], list_dim[2], tau_value, bias)
    dim_found = D - dim_trace_lie_algebra(vector_fields, verbose=0)
    assert dim_found == 0, "dim_found = {}, n_polynomial_functions = {}, for list_dim {}".format(dim_found, dim_poly, list_dim)

100%|███████████████████████████████████████████| 20/20 [12:09<00:00, 36.49s/it]


Again, in this mixed mirror-euclidean geometry with momentum, we obtain that there is zero conservation law!